In [412]:
import pandas as pd
import numpy as np
df = pd.read_csv('dataset2.csv')

print(df.head(), df.shape)

   age     sex     bmi  children smoker     region      charges
0   19  female  27.900         0    yes  southwest  16884.92400
1   18    male  33.770         1     no  southeast   1725.55230
2   28    male  33.000         3     no  southeast   4449.46200
3   33    male  22.705         0     no  northwest  21984.47061
4   32    male  28.880         0     no  northwest   3866.85520 (1337, 7)


In [413]:
# Dummy modele

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import FunctionTransformer
import math

# separation des features et de la variable cible
X = df.drop('charges', axis=1)
Y = df['charges']
print(f'''verif des dimensions X et Y
      X (dataset sans la variable cible): {X.shape}
      Y (la variable cible) : {Y.shape}''')

# Y_log = np.log1p(Y)

# division du dataset en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
print(f''' verif du split 80 20 
80% du dataset : X train -> {X_train.shape}, Y train -> {y_train.shape}
20% du dataset : X test -> {X_test.shape}, Y test -> {y_test.shape}''')

# # Pipeline pour appliquer les transformations en séquence
# num_pipeline = Pipeline([
#     ('log', FunctionTransformer(np.log1p, validate=True)),
#     ('scaler', StandardScaler())
# ])

# Preprocessing avec ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['age', 'bmi', 'children']),
        ('cat', OneHotEncoder(), ['region', 'sex', 'smoker'])
    ]
)

verif des dimensions X et Y
      X (dataset sans la variable cible): (1337, 6)
      Y (la variable cible) : (1337,)
 verif du split 80 20 
80% du dataset : X train -> (1069, 6), Y train -> (1069,)
20% du dataset : X test -> (268, 6), Y test -> (268,)


In [414]:
from sklearn.dummy import DummyRegressor

# création du pipeline dummy
pipeline_dummy = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', DummyRegressor(strategy='mean'))])


# eviter data leakage -> entraîner le pipeline sur les données d'entraînement puis predire
# sur l'ensemble de test avec le meme pipeline
pipeline_dummy.fit(X_train, y_train)
#puis predire y sur l'ensemble de test avec le meme pipeline
y_pred = pipeline_dummy.predict(X_test)
#print(y_pred_dummy)


#test avec les differentes métriques pour le modele dummy : 
#comparaison du y prédit avec le y de test
mse = mean_squared_error(y_test, y_pred)

print(f'Mean Squared Error du modèle dummy: {mse}')

r2 = r2_score(y_test, y_pred)

print(f'Coefficient of Determination R² du modèle dummy: {r2}')

rmse = math.sqrt(mse)
print(f'Root Mean Squared Error (RMSE) du modèle dummy: {rmse}')

Mean Squared Error du modèle dummy: 185298141.68739662
Coefficient of Determination R² du modèle dummy: -0.008391982350864469
Root Mean Squared Error (RMSE) du modèle dummy: 13612.426003009037


In [415]:
# Calculez le RMSE (la racine carrée du MSE) pour obtenir une idée de l'erreur en termes des unités d'origine de vos données. 
# Comparez le RMSE à la moyenne des 'charges' (df['charges'].mean()) pour évaluer l'erreur relative.

# RMSE plus élevé que la moyenne : le modèle a du mal à capturer la variation et les tendances dans les données. 
# Il prédit généralement des valeurs qui sont relativement éloignées des valeurs réelles.

# Erreur systématique :  le modèle a tendance à sous-estimer ou surestimer systématiquement les valeurs cibles.

# Modèle insuffisamment performant : Un RMSE élevé peut également indiquer que le modèle que vous utilisez n'est pas suffisamment complexe 
# pour capturer les relations présentes dans les données. Cela peut signifier que le modèle ne tient pas compte de toutes les caractéristiques
# pertinentes ou que la stratégie de prédiction choisie (comme la moyenne) n'est pas adaptée aux données.

# En général, un bon modèle de régression devrait avoir un RMSE inférieur à la moyenne de la variable cible, 
# car cela indiquerait que le modèle est capable de faire des prédictions qui se rapprochent des valeurs réelles en moyenne. 
# L'objectif est de minimiser le RMSE pour obtenir des prédictions précises.

print(df['charges'].mean())
print(rmse)

#donc on en résume que le modele dummy est tres mauvais, et c'est normal car le modele dummy 
#en général, le modèle dummy est un modèle très simple et naïf qui est utilisé comme baseline 
#pour évaluer la performance d'autres modèles plus sophistiqués.

13279.121486655948
13612.426003009037


donc on en résume que le modele dummy est tres mauvais, et c'est normal car le modele dummy <br>
est un modèle très simple et naïf qui est utilisé comme baseline pour évaluer la performance d'autres modèles plus sophistiqués

In [416]:
# AVEC LE Modele de regression LASSO


from sklearn.linear_model import Lasso, Ridge
from sklearn.model_selection import GridSearchCV


# création du pipeline de prétraitement et du modèle dummy
pipeline_lasso = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regression', Lasso())])


#test de différents hyperparam d'alpha pour le lasso
param_grid_lasso = {
    'regression__alpha': [0.1, 1, 10, 100]  
}

### grid search
grid_lasso = GridSearchCV(pipeline_lasso, param_grid_lasso, cv=5)


# eviter data leakage -> entraîner le pipeline sur les données d'entraînement 
grid_lasso.fit(X_train, y_train)
#puis predire y sur l'ensemble de test avec le meme pipeline
y_pred = grid_lasso.predict(X_test)
# print(y_pred)
score = grid_lasso.score(X_train, y_train)
print(score)
#test des les differents metriques sur modele lasso
#comparaison du y prédit avec le y de test
mse = mean_squared_error(y_test, y_pred)

print(f'Mean Squared Error du modèle lasso: {mse}')

r2 = r2_score(y_test, y_pred)

print(f'Coefficient of Determination R² du modèle lasso: {r2}')

rmse = math.sqrt(mse)
print(f'Root Mean Squared Error (RMSE) du modèle lasso: {rmse}')



0.7298809245190985
Mean Squared Error du modèle lasso: 35640200.074576505
Coefficient of Determination R² du modèle lasso: 0.8060461282703295
Root Mean Squared Error (RMSE) du modèle lasso: 5969.941379492474


In [417]:
print(df['charges'].mean())
print(rmse)


13279.121486655948
5969.941379492474


In [418]:
# Modele de regression RIDGE

# création du pipeline de prétraitement et du modèle dummy
pipeline_ridge = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regression', Ridge())])


#test de différents hyperparam d'alpha pour le lasso
param_grid_ridge = {
    'regression__alpha': [0.01, 0.1, 1, 10, 100]  
}

### grid search
grid_ridge = GridSearchCV(pipeline_ridge, param_grid_ridge, cv=5)


# eviter data leakage -> entraîner le pipeline sur les données d'entraînement 
grid_ridge.fit(X_train, y_train)
#puis predire y sur l'ensemble de test avec le meme pipeline
y_pred = grid_ridge.predict(X_test)
# print(y_pred)
score = grid_ridge.score(X_train, y_train)
print(score)

#test des les differents metriques sur modele lasso
#comparaison du y prédit avec le y de test
mse = mean_squared_error(y_test, y_pred)

print(f'Mean Squared Error du modèle ridge: {mse}')

r2 = r2_score(y_test, y_pred)

print(f'Coefficient of Determination R² du modèle ridge: {r2}')

rmse = math.sqrt(mse)
print(f'Root Mean Squared Error (RMSE) du modèle ridge: {rmse}')



Mean Squared Error du modèle ridge: 35572594.477132455
Coefficient of Determination R² du modèle ridge: 0.8064140377474771
Root Mean Squared Error (RMSE) du modèle ridge: 5964.2765258774225


In [419]:
print(df['charges'].mean())
print(rmse)


13279.121486655948
5964.2765258774225


C'est plutot bien car on a un rmse plus bas que la moyenne pour le lasso et le ridge <br>
l'objectif est de minimiser le RMSE pour obtenir des prédictions précises <br>
r2 a 0,8 donc il predit la bonne réponse à 80% pour les deux modeles de regression <br>

Les resultats des deux modeles de regression lasso et ridge sont presques les memes et ils sont netement mieux que le dummy modele